In [278]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

np.random.seed(42)

In [279]:
def generate(filename):
    df = pd.read_csv(filename,delimiter='\s+',header=None)
    X = df[df.columns[:-1]].values
    y = df[df.columns[-1]].values
    
    num_classes = len(np.unique(y))
    N = len(X)
    
    Y = np.zeros((N,num_classes))
    
    for i in range(N):
        Y[i][y[i]-1] = 1
    
    return X,Y

In [280]:
X_train,Y_train = generate('trainNN.txt')
X_test, Y_test  =  generate('testNN.txt')

In [281]:
class L_Layer_NN():
    def __init__(self,k,lr=0.25,max_epoch=1000):
        self.k = k
        self.L = len(self.k)-1
        self.weights = []
        self.lr = lr
        self.max_epoch = max_epoch
        
    def init_weights(self):
        for i in range(1,len(self.k)):
            self.weights.append(np.random.uniform(0,1,(self.k[i],self.k[i-1]+1)))
    def sigmoid(self,z):
        return 1/(1 + np.exp(-z))

    def derivative(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
            
    def feed_forward(self,x_i):
        self.y = []
        self.v = []
        
        self.y.append(x_i)
        
        for r in range(self.L):
            v_r = np.dot(self.weights[r],self.y[r])
            self.v.append(v_r)
            y_r = self.sigmoid(v_r)
            y_r = np.insert(y_r,0,values=1,axis=0)
            self.y.append(y_r)
    
    def backpropagation(self,y_true):
        self.delta = [0]*self.L
        self.y_hat = self.y[-1]
        self.delta[self.L-1] = np.multiply(self.y_hat[1:]-y_true,self.derivative(self.v[self.L-1]))    
        
        for r in reversed(range(0,self.L-1)):
            e_r = np.dot(self.delta[r+1],self.weights[r+1][:,1:])
            self.delta[r] = np.multiply(e_r,self.derivative(self.v[r]))
    
    def update_weights(self):
        self.y = self.y[:-1]
        for r in range(self.L):
            self.weights[r] = self.weights[r] - self.lr*((self.delta[r].reshape(-1,1))*self.y[r])
    
    def calculate_cost(self,y_hat,y_true):
        return 0.5*np.sum((y_hat-y_true)**2,axis=0)
    
    def fit(self,X,Y):
        self.init_weights() ## initialize weights
        X = np.insert(X,0,values=1,axis=1) ## appending 1 to all x vectors
        
        for epoch in range(self.max_epoch):
            total_cost = 0
            for (x_i,y_i) in zip(X,Y):
                self.feed_forward(x_i)
                self.backpropagation(y_i)
                self.update_weights()
                total_cost += self.calculate_cost(self.y_hat[1:],y_i)
            print("Epoch: ",epoch,total_cost)
            if total_cost < 5:
                break
        return
    
    def predict(self,X,Y):
        correct = 0
        sample_number = 0
        
        X = np.insert(X,0,values=1,axis=1)
        
        for (x_i,y_i) in zip(X,Y):
            self.feed_forward(x_i)
            sample_number += 1
            
            predicted_class = np.argmax(self.y[-1][1:])
            actual_class = np.argmax(y_i)
            
            if predicted_class == actual_class:
                correct += 1
            else:
                print(sample_number,x_i,y_i,actual_class+1,predicted_class+1)
        
        return correct/len(X)

In [282]:
min_max_scaler = MinMaxScaler()  # min max scaler
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.fit_transform(X_test)

input_neurons = X_train.shape[1]
output_neurons = Y_train.shape[1] 

model = L_Layer_NN([input_neurons,3,2,output_neurons],lr=1)
model.fit(X_train,Y_train)

Epoch:  0 194.24704028385736
Epoch:  1 149.06985364407905
Epoch:  2 114.91484728610091
Epoch:  3 83.11638804182851
Epoch:  4 59.80908657196476
Epoch:  5 50.00194596025624
Epoch:  6 41.82773707579726
Epoch:  7 33.28253490236738
Epoch:  8 27.151673752327678
Epoch:  9 23.200938084815483
Epoch:  10 20.479680218579244
Epoch:  11 18.18441502997965
Epoch:  12 16.13082347019566
Epoch:  13 14.46809662943353
Epoch:  14 13.008145851915048
Epoch:  15 11.168075587210643
Epoch:  16 10.221813736240094
Epoch:  17 9.067205572121953
Epoch:  18 8.200786269596776
Epoch:  19 6.919109521313854
Epoch:  20 7.294187022130062
Epoch:  21 5.7464280400354255
Epoch:  22 5.445498174037877
Epoch:  23 5.089425880275462
Epoch:  24 4.711045083410885


In [283]:
model.predict(X_train,Y_train)

1.0

In [284]:
model.predict(X_test,Y_test)

1.0